In [111]:
#libraries
from bs4 import BeautifulSoup, element
import urllib
import requests
import pandas as pd
import numpy as np
import lyricsgenius
import re

#NLP stuff
#Tokenization
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords #look into if I should keep this or not
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer


In [112]:
genius = lyricsgenius.Genius("rPVs-pFT7GhBfTxhpu-ISnNCcvCsbRt8wIwhkkEXovrADgSBQfkndQW7Ge22R5Ts")
song = genius.search_song("Luther", "Kendrick Lamar")
lyrics = song.lyrics
print(lyrics)

Searching for "Luther" by Kendrick Lamar...
Done.
300 ContributorsTranslationsPortuguêsFrançaisEspañolTürkçeItalianoPolskiDeutschРусский (Russian)Українськаluther Lyrics“luther” is the third track on Kendrick Lamar’s sixth studio album, GNX and was later released as its third single. It marks his  sixth collaboration with his longtime friend SZA.

The title comes from the… Read More [Intro]
If this world were mine

[Verse 1: Kendrick Lamar]
Hey, Roman numeral seven, bae, drop it like it's hot
If this world was mine, I'd take your dreams and make 'em multiply
If this world was mine, I'd take your enemies in front of God
Introduce 'em to that light, hit them strictly with that fire
Fah-fah, fah-fah-fah, fah-fah, fah
Hey, Roman numeral seven, bae, drop it like it's hot
If this world was mine, I'd take your dreams and make 'em multiply
If this world was mine, I'd take your enemies in front of God
Introduce 'em to that light, hit them strictly with that fire
It's a vibe, do your dance, let 

C:\Users\Monado\AppData\Roaming\Python\Python312\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


# Cleaning Lyrics

In [113]:
#Data Cleaning/Processing

def clean_lyrics(raw_lyrics):
    #Get rid of information shoved at beginning of string leaving only the lyrics of the song
    starter = ']' #This always finds the first closed bracket which is always the closing bracket for '[Intro]'
    position = raw_lyrics.find(starter)
    lyrics = raw_lyrics[position+1:]

    # Remove section headers like [Chorus], [Verse 1], etc.
    lyrics = re.sub(r"\[.*?\]", "", lyrics)

    # Remove extra whitespace, newlines
    lyrics = re.sub(r"\n+", " ", lyrics)
    lyrics = re.sub(r"\s+", " ", lyrics)

    # Optionally remove punctuation
    lyrics = re.sub(r"[^\w\s']", "", lyrics)

    # Lowercase
    lyrics = lyrics.lower()

    return lyrics


In [114]:
cleaned_lyrics = clean_lyrics(lyrics)
print(cleaned_lyrics)

 if this world were mine hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire fahfah fahfahfah fahfah fah hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire it's a vibe do your dance let 'em watch she a fan he a flop they just wanna kumbaya nah in this world concrete flowers grow heartache she only doin' what she know weekends get it poppin' on the low better days comin' for sure if this world were if it was up to me i wouldn't give these nobodies no sympathy i'd take away the pain i'd give you everything i just wanna see you win wanna see if this world were mine it go in when you out ride it do it real slow slide baby y

# Tokenization/Lemmatization/Filtering stopwords

In [115]:
# Tokenize into words
words = word_tokenize(cleaned_lyrics)

# Tokenize into sentences
sentences = sent_tokenize(cleaned_lyrics) #this one doesn't do anything 

In [116]:
print(words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 

In [117]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

In [118]:
print(lemmatized_words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 'kumbaya',

In [119]:
#Consider if I should keep this or not. Lose of some words may affect models effectivness
stop_words = set(stopwords.words("english"))

filtered_words = [word for word in lemmatized_words if word not in stop_words]

In [120]:
print(filtered_words)

['world', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', "'s", 'vibe', 'dance', 'let', "'em", 'watch', 'fan', 'flop', 'wan', 'na', 'kumbaya', 'nah', 'world', 'concrete', 'flower', 'grow', 'heartache', 'doin', "'", 'know', 'weekend', 'get', 'poppin', "'", 'low', 'better', 'day', 'comin', "'", 'sure', 'world', 'wa', 'would', "n't", 'give', 'nobody', 'sympathy', "'d", 'take', 'away', 'pain', "'d", 'give', 'everything', 'wan', 'na', 'see', 'win', 'wan', 'na', 'see', 'world

# NLP

In [137]:
#Sentiment Analysis
sia = SentimentIntensityAnalyzer()
scores = sia.polarity_scores(cleaned_lyrics)

print(scores)
# {'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.5719}

{'neg': 0.09, 'neu': 0.774, 'pos': 0.135, 'compound': 0.9697}


In [138]:
#Determine setniment score based off compount

if scores['compound'] >= 0.05:
    sentiment = 'positive'
elif scores['compound'] <= -0.05:
    sentiment = 'negative'
else:
    sentiment = 'neutral'

In [139]:
print(sentiment)

positive


In [142]:
#Identify which words are most unique

# Assume `lyrics_list` is a list of cleaned lyrics from multiple songs
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(filtered_words)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Top keywords for a song:

def get_top_keywords(tfidf_vector, feature_names, top_n=10):
    # vector_dense = tfidf_vector.toarray().flatten()
    # sorted_indices = np.argsort(vector_dense)[::-1]  # Descending order
    # top_keywords = [(feature_names[i], vector_dense[i]) for i in sorted_indices[:top_n] if vector_dense[i] > 0]
    # return top_keywords
    sorted_indices = np.argsort(tfidf_vector.toarray()[0])[::-1]
    top_keywords = [feature_names[i] for i in sorted_indices[:top_n]]
    return top_keywords

In [143]:
get_top_keywords(tfidf_matrix[0], feature_names)

['world',
 'flower',
 'god',
 'grow',
 'heartache',
 'hey',
 'hit',
 'home',
 'hot',
 'introduce']

In [144]:
print(tfidf_matrix)

  (0, 87)	1.0
  (2, 27)	1.0
  (3, 57)	1.0
  (4, 46)	1.0
  (5, 60)	1.0
  (6, 2)	1.0
  (7, 14)	1.0
  (8, 37)	1.0
  (10, 30)	1.0
  (11, 87)	1.0
  (12, 79)	1.0
  (16, 12)	1.0
  (17, 42)	1.0
  (18, 15)	1.0
  (19, 43)	1.0
  (20, 87)	1.0
  (21, 79)	1.0
  (25, 16)	1.0
  (27, 24)	1.0
  (28, 31)	1.0
  (29, 15)	1.0
  (30, 36)	1.0
  (31, 28)	1.0
  (32, 68)	1.0
  (34, 18)	1.0
  :	:
  (210, 58)	1.0
  (211, 38)	1.0
  (214, 75)	1.0
  (216, 86)	1.0
  (218, 67)	1.0
  (221, 59)	1.0
  (222, 41)	1.0
  (225, 51)	1.0
  (226, 41)	1.0
  (228, 73)	1.0
  (229, 77)	1.0
  (230, 77)	1.0
  (231, 1)	1.0
  (232, 84)	1.0
  (234, 48)	1.0
  (236, 40)	1.0
  (237, 3)	1.0
  (238, 10)	1.0
  (239, 6)	1.0
  (241, 70)	1.0
  (242, 32)	1.0
  (244, 6)	1.0
  (246, 3)	1.0
  (247, 10)	1.0
  (248, 87)	1.0
